## YaSpeller
**Проверка орфографии**

In [8]:
import requests
from pyaspeller import Word

In [4]:
check_wrong = Word('Кампьютер')
print(check_wrong.correct)
print(check_wrong.variants)

False
['Компьютер']


In [9]:
check_correct = Word('Стол')
print(check_correct.correct)

True


## YaDictionary
**Поиск синонимов**

In [91]:
import sys 
import config
sys.path.append('portYaDict.py')
from portYaDict import YandexDictionary

In [92]:
def search_syn(word):
    
    # Получаем ответ API на запрос по слову
    dictionary = YandexDictionary(config.API_DICT_KEY)
    output = dictionary.lookup(word, 'ru', 'ru')
    
    # Фильтруем (очищаем) данные
    russian_letters = set("ёйцукенгшщзхъфывапролджэячсмитьбю ")
    russian_text = ""
    for letter in output:
        if letter in russian_letters:
            russian_text += letter    
        else:
            russian_text += ","
    
    # Массив синонимов, возвращаем первые 5
    synonymus = []
    synonymus = list(filter(lambda x: len(x.split(" ")) == 1 and len(x) > 0, list(russian_text.split(','))))
    
    return synonymus[0:5]

In [93]:
search_syn("город")

['город', 'сити', 'мегаполис', 'г', 'град']

## Compare texts

### With difflib
**Построение векторов**

In [35]:
import difflib, time

In [52]:
def similarity_diff(text1, text2):
    start_time = time.time()
    
    normalized1 = text1.lower()
    normalized2 = text2.lower()
    matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
    
    work_time = round(time.time() - start_time)
    print("Work time: {0}".format(work_time))
    return matcher.ratio()

In [53]:
sim_diff = similarity_diff("Мясо гигантских тараканов станет вкусной и недорогой альтернативой говядине",
                      "Гигантское мясо тараканов станет говядине недорогой и вкусной альтернативой")
print(sim_diff)

Work time: 0
0.6933333333333334


### With sklearn
**Тоже построение векторов (только лучше) + косинусное расстояние**

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
import os

In [55]:
def similarity_sklearn(text1, text2):
    start_time = time.time()
    
    vect = TfidfVectorizer(min_df=1)
    tfidf = vect.fit_transform([text1, text2])

    work_time = round(time.time() - start_time)
    print("Work time: {0}".format(work_time))
    return(tfidf * tfidf.T).A[0][1]

In [56]:
sim_sklearn = similarity_sklearn("Мясо гигантских тараканов станет вкусной и недорогой альтернативой говядине",
                                 "Гигантское мясо тараканов станет говядине недорогой и вкусной альтернативой")
print(sim_sklearn)

Work time: 0
0.7799154245579976


## Text clustering
**Сравнение векторов**

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

documents = ["Хлеб — хлебобулочное изделие без начинки",
             "Для приготовления хлеба употребляют пшеничную и ржаную муку, реже — кукурузную, ячменную и другие",
             "Хлеб можно есть отдельно, однако нередко его едят со сливочным, арахисовым или подсолнечным маслом",
             "Хлеб — один из старейших приготавливаемых продуктов, появившийся ещё в неолите",
             "Вода является хорошим сильнополярным растворителем.",
             "Исключительно важна роль воды в глобальном кругообороте вещества и энергии",
             "Это солёная вода, непригодная для сельского хозяйства и питья",
             "Пресная же вода находится в основном в ледниках"]

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Y = vectorizer.transform(["Люблю пить воду"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["Ем хлеб с утра и вечером с маслом"])
prediction = model.predict(X)
print(prediction)
 

Top terms per cluster:
Cluster 0:
 хлеб
 изделие
 без
 хлебобулочное
 начинки
 старейших
 продуктов
 один
 неолите
 приготавливаемых
Cluster 1:
 вода
 для
 является
 сильнополярным
 хорошим
 растворителем
 основном
 пресная
 же
 находится


Prediction
[1]
[0 1 0 0 1 1 1 1]
